In [ ]:
import numpy as np
import pandas as pd
# import cvxpy as cvx
from scipy.optimize import minimize
import sympy as sp
import matplotlib.pyplot as plt
import sympy as sp
from functools import reduce
from mpl_toolkits import mplot3d
from scipy.ndimage.interpolation import shift

%matplotlib notebook

# Implementar algoritmos 5.1, 5.2, algo do prob. 5.15

### 5.7

$$ f(x) = (x_1^2 + x_2^2 - 1)^2 + (x_1 + x_2 -1)^2 $$

In [ ]:
def loss1(x):
    return (x[0]**2 + x[1]**2 -1)**2 + (x[0] + x[1] -1)**2

nbDims = 2
symbolVec = sp.symbols('a0:%d'%nbDims)
cost = sp.lambdify(symbolVec, loss1(symbolVec), modules=['numpy'])
gradCost = sp.lambdify(symbolVec,[sp.diff(loss1(symbolVec),var) for var in (symbolVec)],'numpy')
hessianCost = sp.lambdify(symbolVec, sp.hessian(loss1(symbolVec),symbolVec),'numpy')

domain = np.linspace(-10, 10,500)
X, Y = np.meshgrid(domain, domain)
Z = cost(X,Y)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X,Y,Z, cmap=plt.cm.coolwarm)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')

ax.set_title('$f(x)=%s$' % sp.latex(loss1(sp.symbols('x y'))));
plt.tight_layout()
# plt.savefig("lista2/relatorio/figs/loss4.eps", 
# #                bbox_inches='tight', 
# #                transparent=True,
#                pad_inches=0)


In [ ]:
def generalDescentMethod(x0, costSymbolic, specificDescentMethod, eps=1e-6, **args):
    
    symbolVec = sp.symbols('a0:%d'%len(x0))
    descentAlgo = specificDescentMethod(symbolVec, costSymbolic, **args)
    nbEvalDict = {}
    params = {}
    step = np.inf
    xHist = [x0]
    fxHist = [np.NaN]
    
    while np.linalg.norm(step) > eps:
        
        step, fx, partialNbEvalDict, params = descentAlgo(x0, params)
        nbEvalDict = { k: nbEvalDict.get(k, 0) + partialNbEvalDict.get(k, 0) for k in set(partialNbEvalDict) }
        x0 += step
        xHist.append(x0)
        fxHist.append(fx)

    return xHist, fxHist, nbEvalDict

In [ ]:
def steepestDescent(symbolVec, costSymbolic, **args):

    cost = sp.lambdify(symbolVec, costSymbolic(symbolVec), modules=['numpy'])
    gradCost = sp.lambdify(symbolVec,[sp.diff(costSymbolic(symbolVec),var) for var in (symbolVec)],'numpy')
    hessianCost = sp.lambdify(symbolVec, sp.hessian(costSymbolic(symbolVec),symbolVec),'numpy')
    lineSearchMethod = args.pop('lineSearchMethod', noLineSearch)

    def steepestDescentAlgo(x0, params):

        dfx = np.asarray(gradCost(*x0))
        direction = -dfx
        t,f, nbEval, newParams = lineSearchMethod(x0, dfx, direction, cost, params, **args)
        step = t*direction

        return step, f, {'nbFuncEval': nbEval, 'nbGradEval':1}, newParams

    return steepestDescentAlgo

In [ ]:
def newtonDescent(symbolVec, costSymbolic, **args):

    cost = sp.lambdify(symbolVec, costSymbolic(symbolVec), modules=['numpy'])
    gradCost = sp.lambdify(symbolVec,[sp.diff(costSymbolic(symbolVec),var) for var in (symbolVec)],'numpy')
    hessianCost = sp.lambdify(symbolVec, sp.hessian(costSymbolic(symbolVec),symbolVec),'numpy')
    lineSearchMethod = args.pop('lineSearchMethod', noLineSearch)
    beta = args.pop('beta', 0.3)

    def basicNewtonAlgo(x0, params):

        dfx = np.asarray(gradCost(*x0))
        Hx = np.asarray(hessianCost(*x0))
        eigVals = np.linalg.eigvals(Hx)
        minVal = np.min(eigVals)
        if np.any(eigVals < 0.1):
            beta = 1.5*np.abs(np.min(eigVals))
            Hx = (Hx + beta*np.eye(Hx.shape[0]))/(1+beta)

        HxInv = np.linalg.inv(Hx)
        direction = -HxInv.dot(dfx)        
        t,f, nbEval, newParams = lineSearchMethod(x0, dfx, direction, cost, params, **args)
        step = t*direction

        return step, f, {'nbFunEval': nbEval, 'nbGradEval':1, 'nbHessEval':1}, newParams

    return basicNewtonAlgo

In [287]:
def gaussNewtonDescent(symbolVec, costSymbolic, **args):

    cost = sp.lambdify(symbolVec, loss2Vec(symbolVec), modules=['numpy'])
    scalarCost = sp.lambdify(symbolVec,loss2Vec(symbolVec).dot(loss2Vec(symbolVec)), modules=['numpy'])
#     gradCost = sp.lambdify(symbolVec,[sp.diff(loss2Vec(symbolVec),var) for var in (symbolVec)],'numpy')
    jacobian = sp.lambdify(symbolVec, loss2Vec(symbolVec).jacobian(symbolVec), modules=['numpy'])
    lineSearchMethod = args.pop('lineSearchMethod', noLineSearch)
    beta = args.pop('beta', 0.3)

    def gaussNewtonAlgo(x0, params):
        
        
        fx = cost(*(x0))
        Jx = np.asarray(jacobian(*x0))
        dfx = 2*Jx.T.dot(fx)
        Hx = 2*Jx.T.dot(Jx)
        
        L,D = hessianCorrectionMatDav(Hx)
        y = - L.dot(dfx)
        direction = L.T.dot(np.linalg.inv(D).dot(y))[:,0]

        t,f, nbEval, newParams = lineSearchMethod(x0, dfx, direction, scalarCost, params, **args)
        step = t*direction

        return step, cost(*(x0+step)), {'nbFunEval': nbEval, 'nbGradEval': dfx.shape[0]}, newParams

    return gaussNewtonAlgo

In [ ]:
def hessianCorrectionMatDav(H):
    n = H.shape[1]
    L = np.zeros(H.shape)
    D = np.zeros(n)
    h00 = 0
    
    if H[0,0] > 0:
        h00 = H[0,0]
    else:
        h00 = 1
    
    for k in range(1,n):
        m = k - 1
        L[m,m] = 1
        if H[m,m] <= 0:
            H[m,m] = h00
        
        for i in range(k,n):
            L[i,m] = - H[i,m]/H[m,m]
            H[i,m] = 0
            for j in range(k,n):
                H[i,j] += L[i,m]*H[m,j]
        
        if H[k,k] > 0 and H[k,k] < h00:
            h00 = H[k,k]
        
    L[-1,-1] = 1
    if H[-1,-1] <= 0:
        H[-1,-1] = h00
    for i in range(n):
        D[i] = H[i,i]

    return L, np.diag(D)

In [ ]:
def noLineSearch(x, dfx, vec, cost, params, **args):

    alphaHat = params.pop('alpha', 1)
    fxk = params.pop('fxk', None)
    nbEval = 0
    if fxk is None:
        fxk = cost(*x)
        nbEval += 1
    
    fHat = cost(*(x + alphaHat*vec))
    dfx2alpha = np.inner(dfx,dfx)*alphaHat
    alpha = (dfx2alpha*alphaHat)/(2*(fHat - fxk + dfx2alpha))
    fx = cost(*(x + alpha*vec))
    
    return alpha, fx, nbEval+2, {'alpha': alpha, 'fxk': fx}
    

In [286]:
def backtrackingLineSearch(x, dfx, vec, cost, params, **args):
    
    domain = args.pop('domain', None)
    fx = args.pop('fx', None)
    alpha = args.pop('alpha', 0.15)
    beta = args.pop('beta', 0.5)
    
    t = 1
    nbEval = 1
    
    if fx is None:
        fx = cost(*x)
        nbEval += 1
    
    if domain is not None:
        while (x + t*vec <= domain[0]) or (domain[1] <= x + t*vec):
            ptiny
            t *= beta

    f = cost(*(x + t*vec))
    
    while f > (fx + alpha*t*np.vdot(dfx,vec)):
        t *= beta
        f = cost(*(x + t*vec))
        nbEval += 1

    return t,f, nbEval, {}

## 5.7

In [ ]:
x0s = np.array(((4.0,4.0), (4.0,-4.0), (-4.0,4.0), (-4.0,-4.0)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('Using algo 5.1')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss1, steepestDescent,
                                                     lineSearchMethod=backtrackingLineSearch,
                                                     alpha=0.01,beta=0.3)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)
    
x0s = np.array(((4.0,4.0), (4.0,-4.0), (-4.0,4.0), (-4.0,-4.0)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('Using algo 5.2')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss1, steepestDescent,
                                                          lineSearchMethod=noLineSearch)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

## 5.8

In [ ]:
x0s = np.array(((4.0,4.0), (4.0,-4.0), (-4.0,4.0), (-4.0,-4.0)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('Using algo 5.2')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss1, steepestDescent,
                                                          lineSearchMethod=noLineSearch)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

## 5.17

In [ ]:
x0s = np.array(((4.0,4.0), (4.0,-4.0), (-4.0,4.0), (-4.0,-4.0)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('Using Hessian modified algo 5.3')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss1, newtonDescent,
                                                          lineSearchMethod=backtrackingLineSearch)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

## 5.20

In [ ]:
def loss2(x):
    return (x[0] + 10*x[1])**2 + 5*(x[2] - x[3])**2 + (x[1] - 2*x[2])**4 + 100*(x[0] - x[3])**4


In [ ]:
def loss2Vec(x):
    return sp.Matrix(((x[0] + 10*x[1]), (sp.sqrt(5)*(x[2] - x[3])), 
                    ((x[1] - 2*x[2])**2), (10*(x[0] - x[3])**2)))

nbDims = 4
symbolVec = sp.symbols('a0:%d'%nbDims)
cost = sp.lambdify(symbolVec, loss2Vec(symbolVec), modules=['numpy'])
gradCost = sp.lambdify(symbolVec,[sp.diff(loss2Vec(symbolVec),var) for var in (symbolVec)],'numpy')

jacobian = sp.lambdify(symbolVec, loss2Vec(symbolVec).jacobian(symbolVec), modules=['numpy'])
print(loss2Vec(symbolVec))
jacobian(*x0).T.dot(jacobian(*x0))
cost(*x0)
(loss2Vec(symbolVec)).dot(loss2Vec(symbolVec))
scalarCost = sp.lambdify(symbolVec,loss2Vec(symbolVec).dot(loss2Vec(symbolVec)), modules=['numpy'])
scalarCost(*x0)

### B. Steepest descent

In [ ]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('Using algo 5.2 w/ line search w/ params alpha=0.15,beta=0.5')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, steepestDescent, eps=1e-6,
                                                          lineSearchMethod=backtrackingLineSearch)
#                                                      alpha=0.01,beta=0.3)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

In [ ]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}
  
print('\n\nUsing algo 5.2 w/ line search w/ params alpha=0.01,beta=0.3')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, steepestDescent, eps=1e-6,
                                                          lineSearchMethod=backtrackingLineSearch,
                                                     alpha=0.01,beta=0.3)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)


In [ ]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('\n\nUsing algo 5.2 w/o line search')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, steepestDescent, eps=1e-6,
                                                          lineSearchMethod=noLineSearch)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

### C. Newton 

In [ ]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}

print('Using algo 5.3 w/ line search w/ params alpha=0.15,beta=0.5')
for x0 in x0s:
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, newtonDescent, eps=1e-6,
                                                          lineSearchMethod=backtrackingLineSearch)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

In [291]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}
   
print('Using algo 5.3 w/ line search w/ params alpha=0.01,beta=0.3')
for x0 in x0s:
    xHist = {}
    fxHist = {}
    nbEvalList = {}
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, newtonDescent, eps=1e-6,
                                                     lineSearchMethod=backtrackingLineSearch,
                                                     alpha=0.01,beta=0.3)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

Using algo 5.3 w/ line search w/ params alpha=0.01,beta=0.3

[-2. -1.  1.  2.]
x min [  4.11797603e-06  -4.11797603e-07   2.89051310e-06   2.89051310e-06]
f(x min) 1.69780832051e-21
nb evals {'nbHessEval': 58, 'nbFunEval': 116, 'nbGradEval': 58}

[ 200. -200.  100. -100.]
x min [  4.01913816e-06  -4.01913816e-07   2.82113727e-06   2.82113727e-06]
f(x min) 1.54058424005e-21
nb evals {'nbHessEval': 87, 'nbFunEval': 174, 'nbGradEval': 87}


In [290]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}
     
print('Using algo 5.3 w/o line search')
for x0 in x0s:
    xHist = {}
    fxHist = {}
    nbEvalList = {}
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, newtonDescent, eps=1e-6,
                                                          lineSearchMethod=noLineSearch)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

Using algo 5.3 w/o line search

[-2. -1.  1.  2.]
x min [-1.56134028 -0.190389    0.95775448  1.24652367]
f(x min) 6247.99546717
nb evals {'nbHessEval': 20, 'nbFunEval': 41, 'nbGradEval': 20}

[ 200. -200.  100. -100.]
x min [ 254.51510172  -77.43336704  101.67907809   43.9214593 ]
f(x min) 202906075604.0
nb evals {'nbHessEval': 27, 'nbFunEval': 55, 'nbGradEval': 27}


Por que ambos algoritmos (Steepest descent e Newton) quando usam o método sem (backtracking) line search falham miseravelmente na busca pelo mínimo global $[0, 0, 0, 0]$ ?

### D. Gauss-Newton

In [289]:
x0s = np.array(((-2.0, -1.0, 1.0, 2.0), (200, -200, 100, -100)))
xHist = {}
fxHist = {}
nbEvalList = {}
   
print('Using algo 5.5 w/ line search w/ params alpha=0.01,beta=0.3')
for x0 in x0s:
    xHist = {}
    fxHist = {}
    nbEvalList = {}
#     print()
#     print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, gaussNewtonDescent, eps=1e-6,
                                                     lineSearchMethod=backtrackingLineSearch,
                                                     alpha=0.01,beta=0.3)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

Using algo 5.5 w/ line search w/ params alpha=0.01,beta=0.3
x min [-0.00451587  0.00045088 -0.00569479 -0.0056964 ]
f(x min) [[ -7.09322008e-06]
 [  3.60180484e-06]
 [  1.40196630e-04]
 [  1.39366976e-05]]
nb evals {'nbFunEval': 264023, 'nbGradEval': 120500}
x min [ 0.00452268 -0.00045156  0.00570336  0.00570498]
f(x min) [[  7.12830423e-06]
 [ -3.61811352e-06]
 [  1.40618787e-04]
 [  1.39783579e-05]]
nb evals {'nbFunEval': 262589, 'nbGradEval': 119960}
